In [1]:
import csv
import os
import pandas as pd

In [5]:
file = os.path.join('data', 'Pitching.csv')
pitching = pd.read_csv(file)
pitching.drop(columns=['stint', 'teamID','lgID'], inplace=True)
pitching.head()

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,2,3,3,2,0,0,78,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,12,15,30,30,30,0,0,792,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,0,0,1,0,0,0,0,3,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,4,16,24,24,22,1,0,639,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,0,1,1,1,1,0,0,27,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


In [22]:
pitching = pitching[pitching.yearID >= 1900]
pitching = pitchingClean.groupby(['playerID', 'yearID'], as_index=False).sum()
pitching.sort_values(by='yearID',ascending=True).head()

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
13331,griffcl01,1900,14,13,30,30,27,4,0,744,...,0.0,5,16.0,0,1018.0,0,126,0.0,0.0,0.0
17784,jonesco01,1900,13,19,39,36,29,3,0,878,...,0.0,5,19.0,0,1268.0,2,185,0.0,0.0,0.0
18815,kitsofr01,1900,15,13,40,30,21,2,4,760,...,0.0,7,9.0,1,1069.0,9,152,0.0,0.0,0.0
23644,mercewi01,1900,13,17,33,29,26,1,0,728,...,0.0,3,20.0,0,1072.0,4,138,0.0,0.0,0.0
32763,seymocy01,1900,2,1,13,7,2,0,0,159,...,0.0,8,10.0,0,273.0,5,54,0.0,0.0,0.0


In [20]:
file = os.path.join('data', 'People.csv')
people = pd.read_csv(file)
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [35]:
pitchingMerge = pitching.merge(people[['playerID','birthYear']], on='playerID', how='left')
pitchingMerge['Age'] = pitchingMerge['yearID'] - pitchingMerge['birthYear']
pitchingMerge['IP'] = (pitchingMerge['IPouts']/3)
pitchingMerge.head()

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,BK,BFP,GF,R,SH,SF,GIDP,birthYear,Age,IP
0,aardsda01,2004,1,0,11,0,0,0,0,32,...,0,61.0,5,8,0.0,1.0,1.0,1981.0,23.0,10.666667
1,aardsda01,2006,3,0,45,0,0,0,0,159,...,0,225.0,9,25,1.0,3.0,2.0,1981.0,25.0,53.000000
2,aardsda01,2007,2,1,25,0,0,0,0,97,...,0,151.0,7,24,2.0,1.0,1.0,1981.0,26.0,32.333333
3,aardsda01,2008,4,2,47,0,0,0,0,146,...,0,228.0,7,32,3.0,2.0,4.0,1981.0,27.0,48.666667
4,aardsda01,2009,3,6,73,0,0,0,38,214,...,0,296.0,53,23,2.0,1.0,2.0,1981.0,28.0,71.333333


In [34]:
pitchingStarters = pitchingMerge[pitchingMerge.IP >= 150]
pitchingStarters.head()

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,BK,BFP,GF,R,SH,SF,GIDP,birthYear,Age,IP
10,aasedo01,1978,11,8,29,29,6,1,0,536,...,0,773.0,0,88,5.0,1.0,12.0,1954.0,24.0,178.666667
11,aasedo01,1979,9,10,37,28,7,1,2,556,...,0,817.0,4,104,8.0,8.0,20.0,1954.0,25.0,185.333333
12,aasedo01,1980,8,13,40,21,5,1,2,525,...,1,761.0,6,83,12.0,9.0,20.0,1954.0,26.0,175.000000
34,abbotgl01,1977,12,13,36,34,7,0,0,613,...,0,864.0,0,111,7.0,4.0,17.0,1951.0,26.0,204.333333
35,abbotgl01,1978,7,15,29,28,8,1,0,466,...,0,689.0,0,99,9.0,3.0,11.0,1951.0,27.0,155.333333
